# Eat Safe, Love

## Notebook Set Up

In [27]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [28]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [29]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [30]:
# review the collections in our database
print(db.list_collection_names())

['establishments']


In [31]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [32]:
# Find the establishments with a hygiene score of 20
query = {'scores.Hygiene': 20}

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)

# Display the first document in the results using pprint
print(count)
pprint(establishments.find_one(query))

0
None


In [33]:
# Convert the result to a Pandas DataFrame
score_20 = pd.DataFrame(establishments.find(query))
# Display the number of rows in the DataFrame
print(f'The number of rows is: {len(score_20)}')
# Display the first 10 rows of the DataFrame
score_20.head(10)

The number of rows is: 0


""


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [34]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query1 = {'LocalAuthorityName': {'$regex': 'London'}, 'RatingValue': {'$gte': 4}}

#use count function to find the number
count = establishments.count_documents(query1)
print(f"Number of establishments in London with RatingValue >= 4: {count}")

result = establishments.find_one(query1)
pprint(result)

Number of establishments in London with RatingValue >= 4: 0
None


In [35]:
# Convert the result to a Pandas DataFrame
rating_4= pd.DataFrame(establishments.find(query1))
# Display the number of rows in the DataFrame
print(f'Number of rows is: {len(rating_4)}')
# Display the first 10 rows of the DataFrame
rating_4.head(10)

Number of rows is: 0


""


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [36]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.490142
longitude = 0.08384
query2 = {
    'RatingValue': 5,
    'geocode.latitude': {'$lte': latitude + degree_search, '$gte': latitude - degree_search},
    'geocode.longitude': {'$lte': longitude + degree_search, '$gte': longitude - degree_search}
}

sort = [('scores.Hygiene', 1)]
limit = 5

# Print the results
for result in establishments.find(query2).sort(sort).limit(limit):
    pprint(result)

In [37]:
# Convert result to Pandas DataFrame
# Convert result to Pandas DataFrame
results= pd.DataFrame(establishments.find(query2).sort(sort).limit(limit))
results.head(10)

""


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [38]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest
match_query = {'$match': {'scores.Hygiene': 0}}
group_query = {'$group': {'_id': '$LocalAuthorityName', 
                          'count': {'$sum': 1}
                         }
              }
sort_query = {'$sort': {'count': -1}}

pipeline = [match_query, group_query, sort_query]

results = list(establishments.aggregate(pipeline))

# Print the number of documents in the result
print(f"Number of documents: {len(results)}")

# Print the first 10 results
pprint(results[0:10])

Number of documents: 0
[]


In [39]:
# Convert the result to a Pandas DataFrame
zero_hygiene = pd.json_normalize(results)

# Display the number of rows in the DataFrame
print(f"Number of rows: {len(zero_hygiene)}")

# Display the first 10 rows of the DataFrame
zero_hygiene.head(10)

Number of rows: 0


""
